In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Building Agents with Gemini and CrewAI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/building_agents_with_crewai.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fiamthuya%2Fgoogle-cloud-workshops%2Fmain%2Fai-agents%2Fgemini%2Fbuilding_agents_with_crewai.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/building_agents_with_crewai.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/building_agents_with_crewai.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Thu Ya Kyaw](https://github.com/iamthuya) |

## Disclaimer: The CrewAI portion of this notebook is adapted from the [Multi AI Agent Systems with CrewAI course](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai)

### Install Vertex AI SDK for Python

Here you will install required packages for this lab.

In [ ]:
! pip install --upgrade --quiet --user google-cloud-aiplatform langchain_google_vertexai crewai

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

Don't worry if you see a notification message like `Your session crashed for an unknown reason.` This is expected as you are shutting down the kernel from the same instance using code.


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your environment (Google Colab only)

If you are running this notebook on Google Colab, uncomment and run the following cell to authenticate your environment. You **don't need to authenticate** if you are using Colab Enterprise.

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

### Initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
import vertexai

PROJECT_ID = "thuya-playground"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

assert PROJECT_ID != "your-project-id", "Please input your project id"
assert LOCATION != "your-location", "Please input your project location"

vertexai.init(project=PROJECT_ID, location=LOCATION)

In the next cell, you will import the required modules from install packages.

In [2]:
from IPython.display import Markdown

from crewai import Agent, Task, Crew
from langchain_google_vertexai import ChatVertexAI

### Testing out Gemini model from LangChain

In the following cell, you will try invoking Gemini model from LangChain library. It is slightly different from using it through Vertex AI SDK.

In [3]:
gemini_flash = ChatVertexAI(
    model_name="gemini-1.5-flash-002",
    project_id=PROJECT_ID,
    location=LOCATION
)

message = "What are the pros and cons of Generative AI Agents?"
response = gemini_flash.invoke(message)

Markdown(response.content)

Generative AI agents, which combine large language models (LLMs) with other AI capabilities to perform complex tasks autonomously, offer exciting possibilities but also present significant challenges. Here's a breakdown of their pros and cons:

**Pros:**

* **Automation of complex tasks:**  Agents can automate tasks that require multiple steps and diverse skills, including planning, execution, and adaptation. This can significantly increase efficiency and productivity in various domains.  Think automating customer service interactions, code generation and debugging, or even managing complex supply chains.
* **Increased creativity and innovation:** By combining different AI models and accessing vast amounts of data, agents can generate novel ideas, designs, and solutions that humans might not conceive. This opens up new avenues for creativity in fields like art, design, and scientific research.
* **Personalized experiences:** Agents can tailor their interactions and outputs to individual users, providing personalized recommendations, assistance, and learning experiences.  This is particularly relevant in areas like education, healthcare, and entertainment.
* **Enhanced accessibility:** Agents can translate languages, provide summaries of complex information, and offer assistance to people with disabilities, making information and services more accessible to a wider audience.
* **Improved decision-making:** By analyzing large datasets and considering multiple factors, agents can aid in making more informed and data-driven decisions across various industries.
* **24/7 availability:** Unlike human agents, AI agents can operate continuously, providing uninterrupted service and support.
* **Cost reduction:**  While initial development can be expensive, in the long run, automated tasks can lead to reduced labor costs and increased efficiency.


**Cons:**

* **Bias and fairness:**  Agents are trained on data, and if that data reflects existing societal biases, the agents will likely perpetuate and even amplify those biases in their outputs. This can lead to unfair or discriminatory outcomes.
* **Lack of explainability and transparency:** The decision-making processes of complex agents can be opaque, making it difficult to understand why an agent made a particular decision. This lack of transparency can hinder trust and accountability.
* **Safety and security risks:** Malicious actors could potentially exploit vulnerabilities in agents to cause harm, spread misinformation, or engage in other harmful activities.  The potential for misuse is a major concern.
* **Job displacement:** The automation capabilities of agents could lead to job displacement in certain sectors, requiring workforce retraining and adaptation.
* **High development costs:**  Building and deploying sophisticated generative AI agents requires significant expertise, resources, and infrastructure.
* **Data dependency:** The performance of agents is heavily reliant on the quality and quantity of the data they are trained on.  Insufficient or biased data can severely limit their effectiveness.
* **Ethical concerns:**  The use of agents raises ethical questions regarding privacy, autonomy, and responsibility.  Determining liability when an agent makes a mistake is a complex legal and ethical challenge.
* **Hallucinations and inaccuracies:** LLMs, a core component of many agents, are prone to generating factually incorrect or nonsensical information, which can lead to errors and misleading outputs from the agent.


In conclusion, generative AI agents offer immense potential benefits across diverse fields, but their development and deployment must be approached cautiously and responsibly. Addressing the ethical, safety, and societal challenges associated with these powerful technologies is crucial to ensure their beneficial and equitable use.


### Get Started

If there are no errors, you have successfully invoked Gemini model from LangChain library. In the following section, you will be creating a crew of agents that can research, write, and edit about a given topic using CrewAI. You will be exploring about creating agents and their tasks, bundling agents into a crew, and running the end-to-end flow using the crew. 

### Creating Agents

To define agents with CrewAI, you will need to provide them a role, goal and backstory. It also important to know that LLMs perform better when they are role playing.

#### Agent: Planner

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash,
)

#### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash
)

#### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=gemini_flash,
)

### Creating Tasks

Here you will define your Tasks, and provide them a description, expected_output and agent.

#### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

#### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

#### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Creating the Crew

In the next cell, you will be creating a crew of agents to perform the researching, writing, and editing tasks in sequential manner.

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    manager_llm=gemini_flash,
    verbose=True,
)

### Starting the Crew

Now you have everything you need to start the workflow. Run the following cell to kickoff the job!

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

Markdown(result.raw)

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
**Content Plan: Understanding the Impact of Artificial Intelligence**

**1. Prioritized Trends, Key Players, and News:**

* **Trends:** Generative AI (e.g., DALL-E 2, Midjourney, Stable Diffusion, ChatGPT), AI-driven drug discovery, advancements in AI ethics and explainability, the rise of edge AI, and the increasing use of AI in cybersecurity.
* **Key Players:** OpenAI, Google (DeepMind), Microsoft, Amazon (AWS), Meta, Nvidia, IBM, and various startups specializing in specific AI applications.  Noteworthy emerging players should be researched closer to publication date to reflect current 

# Decoding the AI Revolution: Trends, Players, and the Future

Artificial intelligence (AI) is rapidly transforming our world, impacting everything from healthcare to art.  Its influence is undeniable, prompting both excitement and apprehension about its future implications. This article explores the key trends, major players, and ethical considerations shaping the trajectory of this transformative technology.


## The AI Landscape: Emerging Trends

The AI field is characterized by rapid innovation. Generative AI, exemplified by tools like DALL-E 2, Midjourney, Stable Diffusion, and ChatGPT, has captured the public imagination with its ability to create realistic images, text, and even music.  However, significant concerns remain regarding copyright issues and the potential for misuse. Responsible development and deployment are critical to mitigating these risks.

Beyond generative AI, advancements in specialized AI applications are revolutionizing various sectors.  In healthcare, AI is accelerating drug discovery and improving diagnostic accuracy. Financial institutions utilize AI for fraud detection and algorithmic trading, while cybersecurity benefits from AI-powered threat detection systems. Edge AI, processing data locally, improves efficiency and privacy, especially for IoT devices and autonomous systems. The increasing focus on AI ethics and explainability aims to address concerns about bias and transparency.


## The Titans of AI: Key Players and Partnerships

The AI landscape is dominated by a mix of established tech giants and innovative startups.  Companies like OpenAI (with its GPT models), Google DeepMind, Microsoft, Amazon (through AWS), Meta, and Nvidia play leading roles, each contributing unique strengths and focusing on different areas of AI development.  Open source initiatives also play a vital role in driving innovation and fostering collaboration.  The competitive nature of this landscape, while fostering innovation, also raises concerns about potential monopolies.

Strategic partnerships and acquisitions are common, accelerating progress and shaping the AI ecosystem.  This dynamic interplay between large corporations and smaller, specialized companies creates a constantly evolving landscape.


## Navigating the Future: Opportunities and Challenges

AI offers immense potential to improve lives, increasing productivity, advancing healthcare, and automating tasks. However, challenges remain.  Widespread job displacement due to automation requires proactive strategies for workforce retraining and adaptation. Algorithmic bias, stemming from biased data, can perpetuate societal inequalities, emphasizing the need for fairness and transparency in AI development.  The potential for misuse of AI, especially in autonomous weapons systems, necessitates careful ethical and security considerations.  Addressing these concerns through robust regulations and ethical guidelines is crucial.


## Conclusion: A Responsible AI Future

AI's transformative power is reshaping our world.  Responsible development and deployment are crucial to harnessing its benefits while mitigating potential risks. Open dialogue, ethical considerations, and proactive regulatory frameworks are essential to ensuring that AI serves humanity's best interests.  A thoughtful and inclusive approach will be critical to shaping a future where AI benefits everyone.


**Call to Action:**  Share your thoughts and perspectives in the comments below.  Learn more about AI ethics from resources like the [link to a relevant resource on AI ethics].

### Try it yourself

Since you already have a crew at this point, switching to another topic is as simiple as just starting a new job with a new topic as input. Try finding some other topics of your interest below.

In [ ]:
topic = "YOUR TOPIC HERE"  # @param {type:"string"}
result = crew.kickoff(inputs={"topic": topic})

Markdown(result.raw)

# Conclusions

You have explored how to create an agent using Gemini and CrewAI.